In [277]:
import requests
import os
import pandas as pd

In [278]:
def get_api_id():
    with open(".env", "r") as f:
        for line in f:
            listing = line.strip().split("=")
            if listing[0] == "DB_API_ID":
                return listing[1]

def get_api_secret():
    with open(".env", "r") as f:
        for line in f:
            listing = line.strip().split("=")
            if listing[0] == "DB_API_KEY":
                return listing[1]


In [309]:
def stations2file():
    # to get list of all stations:
    base_url="https://apis.deutschebahn.com/db-api-marketplace/apis/ris-stations/v1/stations?limit=6000"
    headers = {
                "DB-Client-ID": get_api_id().replace("\'", ""),
                "DB-Api-Key": get_api_secret().replace("\'", ""),
                "accept": "application/vnd.de.db.ris+json"
            }

    response = requests.get(base_url, headers=headers)
    if response.status_code == 200:
        stations_data=pd.read_json(response.text)
        list_df =[stations_data['stations'][i] for i in range(stations_data['stations'].shape[0])]
        df_stations = pd.DataFrame.from_dict(list_df)
        rows = []
        # appending rows
        for data in list_df:
            data_row1 = data['address']
            data_row1['name'] = data['names']['DE']['name']
            data_row1['stationID']=data['stationID']

            if('position' in data.keys()):
                data_row1['latitude'] = data['position']['latitude']
                data_row1['longitude'] = data['position']['longitude']
            rows.append(data_row1)
        dfnew = pd.DataFrame(rows)
    else:
        print(f"Failed to fetch data")
        dfnew=[]

    dfnew.drop(columns=['street','houseNumber','postalCode']).to_json(r"./data/cities_forTimetable.json")
    return 0

In [310]:
stations2file()

C:\Users\ricat\AppData\Local\Temp\ipykernel_1960\4249492940.py:12: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stations_data=pd.read_json(response.text)


0